# Annex A Execution

A notebook implementing the execution rules from KerML Annex A with PyMBE.

In [1]:
import json
import pymbe.api as pm

import copy

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import *

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicity, \
                                    get_upper_multiplicity, \
                                    does_behavior_have_write_features, \
                                    get_most_specific_feature_type, \
                                    has_type_named, \
                                    get_effective_lower_multiplicity, \
                                    get_feature_bound_values

from uuid import uuid4

## Key Helpers for the Algorithm

These helpers are yet to be implemented in the core of the Python tool and thus need to be more spelled out.

### Check for Connectors to Features

In [2]:
def is_feature_connected(feature):
    print(f"...Inspecting {feature.declaredName} for connector references.")
    if hasattr(feature, "reverseReferenceSubsetting"):
        print(f"...Found link to connector end {feature.reverseReferenceSubsetting[0]}")
        return True
    else:
        print(f"...Found no reverse edge outgoing to connector end.")
        return False

### Identify Important Metatypes

In [3]:
def feature_metas():
    return {'Feature', 'Step'}

def classifier_metas():
    return {'Classifier', 'Behavior', 'Structure'}

def assoc_metas():
    return {'Association'}

def connector_metas():
    return {'Connector', 'Succession'}

def datatype_metas():
    return {'DataType'}

### Print atoms as if in KerML Files

In [4]:
def serialize_kerml_atom(atom_to_print):
    
    atom_string = "#atom\n"
    
    if hasattr(atom_to_print, "throughUnioning"):
        atom_string = ""
    
    if atom_to_print._metatype == "Classifier":
        atom_string = atom_string + "classifier "
    elif atom_to_print._metatype == "Behavior":
        atom_string = atom_string + "behavior "
    elif atom_to_print._metatype == "Association":
        atom_string = atom_string + "association "
    elif atom_to_print._metatype == "Succession":
        atom_string = atom_string + "succession "
    elif atom_to_print._metatype == "Structure":
        atom_string = atom_string + "struct "
    elif atom_to_print._metatype == "DataType":
        atom_string = atom_string + "datatype "
    
    atom_string = atom_string + atom_to_print.declaredName
    
    if len(atom_to_print.throughSubclassification) > 0:
        atom_string = atom_string + " specializes " + \
            ", ".join([sub.basic_name for sub in atom_to_print.throughSubclassification])
        
    if hasattr(atom_to_print, "throughUnioning"):
        atom_string = atom_string + " unions " + \
            ", ".join([sub.basic_name for sub in atom_to_print.throughUnioning])
    
    if len(atom_to_print.throughFeatureMembership) > 0:
        atom_string = atom_string + " {\n"
        for feature in atom_to_print.throughFeatureMembership:
            if feature._metatype == "Feature":
                atom_string = atom_string + "   feature "
            elif feature._metatype == "Step":
                atom_string = atom_string + "   step "
            elif feature._metatype == "Connector":
                atom_string = atom_string + "   connector "
            elif feature._metatype == "Succession":
                atom_string = atom_string + "   succession "
                
            if hasattr(feature, "throughFeatureTyping"):
                atom_string = atom_string + feature.basic_name + " : " + feature.throughFeatureTyping[0].basic_name
            else:
                atom_string = atom_string + feature.basic_name
                
            if len(feature.throughRedefinition) > 0:
                atom_string = atom_string + " redefines " + \
                    ", ".join([sub.basic_name for sub in feature.throughRedefinition])
                
            atom_string = atom_string + ";\n"
        atom_string = atom_string + "}\n"
    elif hasattr(atom_to_print, "throughEndFeatureMembership"):
        atom_string = atom_string + " {\n"
        for feature in atom_to_print.throughEndFeatureMembership:
            if feature._metatype == "Feature":
                atom_string = atom_string + "   end feature "
            elif feature._metatype == "Step":
                atom_string = atom_string + "   step "
            elif feature._metatype == "Connector":
                atom_string = atom_string + "   connector "
                
            if hasattr(feature, "throughFeatureTyping"):
                atom_string = atom_string + feature.basic_name + " : " + feature.throughFeatureTyping[0].basic_name
            else:
                atom_string = atom_string + feature.basic_name
                
            if len(feature.throughRedefinition) > 0:
                atom_string = atom_string + " redefines " + \
                    ", ".join([sub.basic_name for sub in feature.throughRedefinition])
                
            atom_string = atom_string + ";\n"
                
        atom_string = atom_string + "}\n"
    else:
        atom_string = atom_string + ";\n"
        
    return atom_string

## Routines for Execution

The following sections are focused on solving the problem of mapping values to KerML types in the model. The approach taken here is to find one legal set of values for types in the model via an approach where the program will walk straight ahead in the model, deriving values as it goes. This approach is called "execution" here.

### Atom Generation Procedure for Steps 1 through 5

In [5]:
def inspect_type_for_atom_rev_2(input_model,
                          package_to_execute,
                          package_to_populate,
                          atom_index,
                          cf,
                          considered_type,
                          bound_features_to_atom_values_dict):
    
    new_ft_classifier = None
    
    type_name = "None"
    if considered_type is not None:
        type_name = considered_type.basic_name
        
    cf_name = cf.basic_name
    
    lower_features = cf.feature
    
    #if cf._id not in values_dict_passed:
        
    new_ft_classifier = build_from_classifier_pattern(
        owner=package_to_populate,
        name=f"My{type_name}{atom_index + 1}",
        model=input_model,
        specific_fields={},
        metatype=considered_type._metatype,
        superclasses=[considered_type]
    )

    #if is_feature_connected(cf):
    if cf._id in bound_features_to_atom_values_dict:
        bound_features_to_atom_values_dict[cf._id].append(new_ft_classifier)
    else:
        bound_features_to_atom_values_dict.update({cf._id: [new_ft_classifier]})

    print(f"Executing step 3a. Creating atom #{atom_index + 1} to be value for {cf_name} and specializing " + \
          f"{type_name}")
    
    if len(lower_features) == 0:
        print(f"...No lower features found for {cf}. Will finish descent here.")
        
    else:
        print(f"...Lower features found for {cf}. Will evaluate it value generation.")
        #for lf in lower_features:
        generate_values_for_type_annex_a(input_model,
                                         package_to_execute,
                                         package_to_populate,
                                         cf,
                                         atom_index,
                                         bound_features_to_atom_values_dict)
    
    return bound_features_to_atom_values_dict

In [6]:
def find_model_existing_values_for_feature(feat, values_dict):
    
    print(f"...Looking to see if {feat} is bound to other feature values")
    
    values_in_dict = []
    feature_values_shared = False
    
    print(f"...Feature values for {feat} are {feat.throughFeatureValue}")
    
    for bound_val in feat.throughFeatureValue:
        if bound_val._metatype == 'FeatureReferenceExpression':
            print(f"...Found a feature value bound to feature {feat}")
            referred_item = bound_val.throughMembership[0]
            feature_values_shared = True
        elif 'Literal' in bound_val._metatype:
            print(f"...Found literal value for {feat}")

    if feature_values_shared:
        print(f"...Taking values from {referred_item} to match to values set for {feat} ({feat._id})")
        try:
            values_in_dict = values_dict[referred_item._id]
            print(f"...Found values {values_in_dict} to match to values set for {feat} ({feat._id})")
            try:
                values_dict[feat._id] = values_dict[feat._id] + values_in_dict
            except KeyError:
                values_dict.update({feat._id: values_in_dict})
        except KeyError:
            pass
        
        # also apply the value to redefined properties
                
    return values_in_dict

In [7]:
def print_values_dictionary(model, values_dict):
    print_string = ""
    for k, v in values_dict.items():
        print_string = print_string + f">>>Key {model.get_element(k)} ({k}) has values {v}\n"
        
    print(print_string)

In [8]:
def embed_values_in_model_atom_classifier_style(
    input_model,
    bound_features_to_atom_values_dict,
    type_to_value,
    new_classifier,
    package_to_populate
):

    for bound_feature_id in bound_features_to_atom_values_dict.keys():
        bound_feature = input_model.get_element(element_id=bound_feature_id)
        if bound_feature._metatype in feature_metas():
            print(f"(Atom style)...Working to connect the feature {bound_feature} to generated types inside atom {new_classifier} via " + \
                  f"covering pattern with values {bound_features_to_atom_values_dict[bound_feature_id]}.")
            if type_to_value in bound_feature.reverseFeatureMembership:
                redefined_bound_feature = apply_covered_feature_pattern(
                    one_member_classifiers=bound_features_to_atom_values_dict[bound_feature_id],
                    feature_to_cover=bound_feature,
                    type_to_apply_pattern_on=new_classifier,
                    model=input_model,
                    new_types_owner=package_to_populate,
                    covering_classifier_prefix="Values for ",
                    covering_classifier_suffix="",
                    redefining_feature_prefix="",
                    redefining_feature_suffix=" (Covered)",
                )
        if bound_feature._metatype in connector_metas():
            print(f"(Atom style)...Working to connect the feature {bound_feature} to generated types inside atom {new_classifier} via " + \
                  f"covering pattern with values {bound_features_to_atom_values_dict[bound_feature_id]}.")
            print(f"...Building covered connector under {new_classifier}")
                    
            redefined_bound_feature = apply_covered_connector_pattern(
                one_member_classifiers=bound_features_to_atom_values_dict[bound_feature_id],
                feature_to_cover=bound_feature,
                type_to_apply_pattern_on=new_classifier,
                model=input_model,
                new_types_owner=package_to_populate,
                covering_classifier_prefix="Values for ",
                covering_classifier_suffix="",
                redefining_feature_prefix="",
                redefining_feature_suffix="",
                metatype=bound_feature._metatype,
                separate_connectors=True
            )

In [9]:
def create_feature_write_performance_atoms(input_model,
                                            bound_features_to_atom_values_dict,
                                            cf_fwp,
                                            package_to_populate):
    
    """
    Create FeatureWritePerformance atoms and the associated timeslices
    """
    
    # find onOccurrence in feature list
    
    features_in_fwp = cf_fwp.feature
    occurrence_bound = None
    accessed_type = None
    
    print(f"...Features found in FWP are {features_in_fwp}")
    
    for cf in features_in_fwp:
        if cf.basic_name == 'onOccurrence' and len(cf.throughRedefinition) > 0:
            print(f"...Found onOccurrence feature")
            occurrence_bound = get_feature_bound_values(cf)[0]
            print(f"...Found a feature value bound to onOccurrence feature which is {occurrence_bound}")
                    
            # expect that onOccurrence has nested features
            
            for cf2 in cf.feature:
                print(f"...Features found in onOccurrence are {cf.feature}")
                if cf2.basic_name == 'startingAt' and len(cf.throughRedefinition) > 0:
                    for cf3 in cf2.feature:
                        if cf3.basic_name == 'accessedFeature' and len(cf.throughRedefinition) > 0:
                            accessed_type = cf3.throughSubsetting[0]
                            print(f"...Found accessedFeature feature bound to {accessed_type}")
    
    # get the thing to timeslice
    
    time_sliced_occurrence = None
    
    try:
        time_sliced_occurrence = bound_features_to_atom_values_dict[occurrence_bound._id]
        print(f"...Occurrence atom for {occurrence_bound} is found as {time_sliced_occurrence}")
    except KeyError:
        # need to make a new classifier here
        print(f"...No occurrence atom for {occurrence_bound} found. Creating a new one.")
        type_to_value = get_most_specific_feature_type(occurrence_bound)
        base_name = type_to_value.basic_name
        new_classifier = build_from_classifier_pattern(
            owner=package_to_populate,
            name=f"My{base_name}",
            model=input_model,
            specific_fields={},
            metatype=type_to_value._metatype,
            superclasses=[type_to_value]
        )
        time_sliced_occurrence = new_classifier
        bound_features_to_atom_values_dict.update({occurrence_bound._id: [time_sliced_occurrence]})
    
    # create time slice for after this FWP is applied
    
    time_slice = None
    need_new_slice = True
    
    try:
        occurrence_atoms = bound_features_to_atom_values_dict[occurrence_bound._id]
        for oa in occurrence_atoms:
            if oa.basic_name == get_most_specific_feature_type(occurrence_bound).basic_name + "TimeSlice":
                print(f"Found previous time slice created for {time_sliced_occurrence}")
                need_new_slice = False
    except KeyError:
        need_new_slice = True
    
    if need_new_slice:
        new_slice = build_from_portion_pattern(
            owner=package_to_populate,
            name_extension="TimeSlice",
            model=input_model,
            classifier_to_be_portioned=get_most_specific_feature_type(occurrence_bound),
            feature_to_be_set=[],
            feature_values=[],
            specific_fields={}
        )
        try:
            bound_features_to_atom_values_dict[occurrence_bound._id].append(new_slice)
        except KeyError:
            bound_features_to_atom_values_dict.update({occurrence_bound._id: [new_slice]})
    
    type_to_value = get_most_specific_feature_type(cf_fwp)
    base_name = cf_fwp.basic_name
    fwp_atom = build_from_classifier_pattern(
            owner=package_to_populate,
            name=f"FeatureWritePerformanceFor{base_name.capitalize()}",
            model=input_model,
            specific_fields={},
            metatype=type_to_value._metatype,
            superclasses=[type_to_value]
        )
    
    bound_features_to_atom_values_dict.update({cf_fwp._id: [fwp_atom]})
    
    return bound_features_to_atom_values_dict

In [10]:
def generate_values_for_type_annex_a(input_model,
                                     package_to_execute,
                                     package_to_populate,
                                     type_to_value,
                                     atom_index,
                                     bound_features_to_atom_values_dict):
    
    """
    Generate values for the given single type using the Annex A rules
    
    bound_features_to_atom_values_dict - pass as a mutable object, update as we go
    """
    
    print(f"Applying Annex A atom algorithm to {type_to_value.basic_name}")
    
    new_classifier = None
    
    if type_to_value._metatype in classifier_metas() and not has_type_named(type_to_value, "FeatureWritePerformance"):
    
        # Step 1 - create new atom from the classifier
        base_name = type_to_value.basic_name
        new_classifier = build_from_classifier_pattern(
            owner=package_to_populate,
            name=f"My{base_name}",
            model=input_model,
            specific_fields={},
            metatype=type_to_value._metatype,
            superclasses=[type_to_value]
        )
        
        print(f"Executing step 1. Working from {base_name} to create {new_classifier.declaredName}")
    
    # use derived property 'feature' to get all features including the inherited ones
    candidate_features = type_to_value.feature
    
    try:
        #print(f"Trying to find metatype for {type_to_value.throughFeatureTyping}")
        featured_meta = type_to_value.throughFeatureTyping[0]._metatype
        print(f"Type {type_to_value.basic_name} is has a type of metatype {featured_meta}.")
        if featured_meta in datatype_metas():
            print(f"Type {type_to_value.basic_name} is typed by a datatype. Bypassing further elaboration.")
            return bound_features_to_atom_values_dict
    except:
        pass
    
    print(f"...Found features {candidate_features} under the type to value {type_to_value.basic_name}.")
    
    # set up for multiple passes on the feature list, will test per pass to determine whether to handle it
    
    # TODO: Instead of this testing approach, gather all features and allocated into sub-collections that
    # handle these passes
    
    passes = ['Non-connector Features', 'FeatureWritePerformances', 'Connector Features']
    
    for pass_number, pass_kind in enumerate(passes):
        
        print(f"Currently on pass {pass_kind} under {type_to_value}.")
        print(f"Current values dict at this step is")
        print_values_dictionary(input_model, bound_features_to_atom_values_dict)
    
        for cf in candidate_features:
            
            if has_type_named(cf, "FeatureWritePerformance") and not pass_kind == 'FeatureWritePerformances':
                continue
            if cf._metatype not in connector_metas() and not pass_kind == 'Non-connector Features':
                if not has_type_named(cf, "FeatureWritePerformance"):
                    continue
            if cf._metatype in connector_metas() and not pass_kind == 'Connector Features':
                continue
            
            # common pre-processing steps
            
            cf_name = cf.basic_name
            
            lm = get_effective_lower_multiplicity(cf)
            
            if lm > -1:
                # need to test multiplicity
                print(f"...Found effective lower multiplicity of {cf_name} ({cf._id}) as {lm}.")
            elif cf._metatype not in connector_metas():
                print(f"...{cf_name} ({cf._id}) has unbounded multiplicity. Skipping.")
                continue
            
            redefining_features = set(cf.reverseRedefinition)
            cf_redefined = False
            
            for rf in redefining_features:
                print(f"...Discovered that {cf} ({cf._id}) is redefined by {rf} ({rf._id})! Skipping.")
                cf_redefined = True
                
            if cf_redefined:
                continue
            
            # Step 2 - find Features with lower multiplicity > 0 that are not connectors
            
            # Look for existing values for the feature or previous atom assignment
            
            values_set_in_model = find_model_existing_values_for_feature(cf, bound_features_to_atom_values_dict)
            
            # Pass-specific core steps
            
            if has_type_named(cf, "FeatureWritePerformance"):
                print(f"Feature {cf} has type of FeatureWritePerformance")
                if pass_kind == 'FeatureWritePerformances':
                    create_feature_write_performance_atoms(
                        input_model,
                        bound_features_to_atom_values_dict,
                        cf,
                        package_to_populate
                    )
            
            elif cf._metatype not in connector_metas() and pass_kind == 'Non-connector Features':
                if len(values_set_in_model) == 0:
                    # Step 3 - create new Atoms to go with the Features
                    
                    used_typ = get_most_specific_feature_type(cf)
                    
                    if used_typ._metatype in datatype_metas():
                        print(f"Executing step 2. Identified {cf_name} ({cf._id}) as a non-connector Feature. No existing values found. " + \
                              f"This is a datatype. Skipping generation of values.")
                        continue
                    else:
                        print(f"Executing step 2. Identified {cf_name} ({cf._id}) as a non-connector Feature. No existing values found. " + \
                              f"Generating {lm} new values specializing type {used_typ}.")
                    
                    for i in range(0, lm):
                        inspect_type_for_atom_rev_2(input_model,
                          package_to_execute,
                          package_to_populate,
                          i,
                          cf,
                          used_typ,
                          bound_features_to_atom_values_dict
                         )
                                
            if cf._metatype in connector_metas() and pass_kind == 'Connector Features':
                print(f"Executing step 4. Identified {cf.basic_name} as a Connector")

                # check the feature ends of the connector
                
                used_typ = get_most_specific_feature_type(cf)
                
                connector_atoms = []

                connector_ends = cf.throughEndFeatureMembership

                lm_end1 = get_effective_lower_multiplicity(connector_ends[0])
                lm_end2 = get_effective_lower_multiplicity(connector_ends[1])

                if lm_end1 == 1 and lm_end2 == 1:
                    print(f"Executing step 5. Identified {cf} as a Connector with 1-to-1 ends")
                    
                    end_connected_mults = [get_effective_lower_multiplicity(connector_ends[i].throughReferenceSubsetting[0]) for i in range(0,2)]
                    
                    # get the multiplicity of the ends
                    #end1_connected_mult = get_effective_lower_multiplicity(connector_ends[0].throughReferenceSubsetting[0])
                    #end2_connected_mult = get_effective_lower_multiplicity(connector_ends[1].throughReferenceSubsetting[0])
                    
                    for i, end_connected_mult in enumerate(end_connected_mults):
                    
                        print(f"...Effective lower multiplicity of {connector_ends[i].throughReferenceSubsetting[0]}, bound to assoc feature {connector_ends[i]}, " + \
                              f"is {end_connected_mult}")

                    # check to see if the already built instances have a finite value
                    
                    print(f"...Looking for connected end feature values filled in during execution.")
                    
                    found_values = []
                    
                    ends_to_process = []

                    if hasattr(cf, "throughEndFeatureMembership"):
                        for cf_ele in cf.throughEndFeatureMembership:
                            ends_to_process.append(cf_ele)
                    if used_typ is not None and hasattr(used_typ, "throughEndFeatureMembership"):
                        for cf_ele in used_typ.throughEndFeatureMembership:
                            ends_to_process.append(cf_ele)
                    
                    for con_end in ends_to_process:
                        bound_feature = con_end.throughReferenceSubsetting[0]
                        if bound_feature._id in bound_features_to_atom_values_dict:
                            print(f"...Found connected end feature values filled in during execution {bound_features_to_atom_values_dict[bound_feature._id]}!")
                            found_values.append(len(bound_features_to_atom_values_dict[bound_feature._id]))
                                                          
                    number_to_make = max(end_connected_mults + found_values)
                    
                    print(f"...Found that number of atoms to make for {cf} is {number_to_make}")

                    feature_value_atoms = []

                    used_typ = None

                    for i in range(0, number_to_make):
                        print(f"Executing step 5b. Creating atom #{i + 1} to be value for {cf}")

                        typ = []
                        used_typ = None
                        used_name = cf.declaredName
                        used_metatype = "Association"
                        
                        elaboration_end = False

                        if hasattr(cf, "throughFeatureTyping"):
                            typ = cf.throughFeatureTyping

                        if len(typ) > 0:
                            used_typ = typ[0]
                            used_name = used_typ.declaredName
                            used_metatype = used_typ._metatype

                        #feature_value_atoms.append(new_ft_classifier)

                        # need to do this for Association types and also nested end features

                        #ends_to_process = []

                        #if hasattr(cf, "throughEndFeatureMembership"):
                        #    for cf_ele in cf.throughEndFeatureMembership:
                        #        ends_to_process.append(cf_ele)
                        #if used_typ is not None and hasattr(used_typ, "throughEndFeatureMembership"):
                        #    for cf_ele in used_typ.throughEndFeatureMembership:
                        #        ends_to_process.append(cf_ele)

                        for con_end in ends_to_process:
                            print(f"...Inspecting {con_end} for connected features.")
                            if len(con_end.throughReferenceSubsetting) > 0:
                                bound_feature = con_end.throughReferenceSubsetting[0]
                                bound_feature_type = bound_feature.throughFeatureTyping

                                print(f"...Found connected feature {bound_feature}.")

                                if len(bound_feature_type) > 0:
                                    feature_used_type = bound_feature_type[0]

                                    if bound_feature._id in bound_features_to_atom_values_dict:
                                        # if not enough atoms exist, make some more
                                        if (i + 1) > len(bound_features_to_atom_values_dict[bound_feature._id]):
                                            print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " + \
                                                f"be value for {con_end} and also {bound_feature} to fill in rest of values.")
                                            
                                            inspect_type_for_atom_rev_2(input_model,
                                                                      package_to_execute,
                                                                      package_to_populate,
                                                                      i,
                                                                      bound_feature,
                                                                      feature_used_type,
                                                                      bound_features_to_atom_values_dict
                                                                     )
                                            
                                    else:
                                        print(f"Executing step 5b (1-to-1 variant). Creating atom #{i + 1} to " + \
                                            f"be value for {con_end} and also {bound_feature}")
                                        
                                        if has_type_named(bound_feature, "FeatureWritePerformance"):
                                            create_feature_write_performance_atoms(
                                                input_model,
                                                bound_features_to_atom_values_dict,
                                                bound_feature,
                                                package_to_populate
                                            )
                                        
                                        else:
                                            inspect_type_for_atom_rev_2(input_model,
                                                                      package_to_execute,
                                                                      package_to_populate,
                                                                      i,
                                                                      bound_feature,
                                                                      feature_used_type,
                                                                      bound_features_to_atom_values_dict            
                                                                     )
                                        

                        if len(ends_to_process) == 2:
                            source_end = ends_to_process[0]
                            target_end = ends_to_process[1]

                            source_bound_feature = source_end.throughReferenceSubsetting[0]
                            target_bound_feature = target_end.throughReferenceSubsetting[0]
                            
                            try:
                                source_atom = bound_features_to_atom_values_dict[source_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {source_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")
                            
                            try:
                                target_atom = bound_features_to_atom_values_dict[target_bound_feature._id][i]
                            except KeyError:
                                raise KeyError(f"...Failed to find atoms for {target_bound_feature} and connector {cf}." + \
                                               f"{bound_features_to_atom_values_dict}")

                            print(f"...Typing atom association ends from {source_atom} to {target_atom} under " + 
                                 f"{used_name}{i + 1} to specialize {[ft.basic_name for ft in cf.throughFeatureTyping]}")
                            
                            # check for typing
                            
                            cn_types = cf.throughFeatureTyping
                            
                            if len(cf.throughFeatureTyping) == 0:
                                pass

                            new_cn_classifier = build_from_binary_assoc_pattern(
                                name=f"{used_name}{i + 1}",
                                source_role_name=connector_ends[0].basic_name,
                                target_role_name=connector_ends[1].basic_name,
                                source_type=source_atom,
                                target_type=target_atom,
                                model=input_model,
                                metatype=used_metatype,
                                owner=package_to_populate,
                                superclasses=cf.throughFeatureTyping,
                                specific_fields={}
                            )
                            
                            if cf._id in bound_features_to_atom_values_dict:
                                bound_features_to_atom_values_dict[cf._id].append(new_cn_classifier)
                            else:
                                bound_features_to_atom_values_dict.update({cf._id: [new_cn_classifier]})
                            
                            connector_atoms.append(new_cn_classifier)
                        
            
    #if type_to_value._metatype in feature_metas():
        # Step 1 - create new atom from the classifier
        
    #    try:
            #print(f"Trying to find metatype for {type_to_value.throughFeatureTyping}")
    #        feature_type = get_most_specific_feature_type(type_to_value)
            #feature_type = type_to_value.throughFeatureTyping[0]
    #        print(f"Found type {feature_type} for {type_to_value}.")
            
    #        base_name = feature_type.basic_name
    #        new_classifier = build_from_classifier_pattern(
    #            owner=package_to_populate,
    #            name=f"My{base_name}",
    #            model=input_model,
    #            specific_fields={},
    #            metatype=type_to_value._metatype,
    #            superclasses=[type_to_value]
    #        )
            
    #        print(f"Executing step 1. Working from {base_name} to create {new_classifier.declaredName}") 
            
    #    except:
    #        print(f"Cannot find the feature type for {type_to_value.basic_name}")
        
    if type_to_value._metatype in datatype_metas():
        print(f"Classifier {type_to_value.basic_name} is a datatype. Bypassing further elaboration.")
        return bound_features_to_atom_values_dict
    
    elif get_most_specific_feature_type(type_to_value) is not None:
        
        try:
            if get_most_specific_feature_type(type_to_value)._metatype in datatype_metas():
                print(f"Classifier {type_to_value.basic_name} is a datatype or typed by one. Bypassing further elaboration.")
                return bound_features_to_atom_values_dict
        except:
            pass
            
    print(f"Current values dict is")
    print_values_dictionary(input_model, bound_features_to_atom_values_dict)
    
    if len(bound_features_to_atom_values_dict.keys()) == 0:
        return {}
    
    if new_classifier == None:
        print(f"...No classifiers created for {type_to_value}")
        try:
            new_classifier = bound_features_to_atom_values_dict[type_to_value._id][atom_index]
            print(f"...Classifier for {type_to_value} found in bound values dict as {new_classifier}")
        except KeyError:
            pass
    else:
        print(f"...Classifier {new_classifier} created for {type_to_value}")
    
    embed_values_in_model_atom_classifier_style(
        input_model,
        bound_features_to_atom_values_dict,
        type_to_value,
        new_classifier,
        package_to_populate
    )
    
    return bound_features_to_atom_values_dict
    
    #print(f"Bound features dict: \n{bound_features_to_atom_values_dict}")

## Atom Metadata Load

Bring up the Atom metadata.

In [11]:
filename = "A-2-Atoms"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

atoms_data = pm.Model.load_from_post_file(json_file)
atoms_data

c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:117: UserWarning: Cannot process Expression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")
c:\users\bjorn cole\documents\github\pymbe\src\pymbe\label.py:117: UserWarning: Cannot process MetadataAccessExpression elements yet!
  warn(f"Cannot process {expression._metatype} elements yet!")


<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-2-Atoms.json)>

## Annex A.3.2 Without Connectors Case

In [12]:
filename = "A-3-2-WithoutConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

without_connectors_data = pm.Model.load_from_post_file(json_file)
without_connectors_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-2-WithoutConnectors.json)>

In [13]:
packages = [ele for ele in without_connectors_data.elements.values() if ele._metatype == 'Package']
packages

[WithoutConnectorsModelToBeExecuted «Package»,
 WithoutConnectorsExecution «Package»]

In [14]:
without_connectors_to_execute_classifiers = \
    [ele for ele in packages[0].throughOwningMembership if ele._metatype == 'Classifier']

In [15]:
packages[0].throughOwningMembership

[fd3d0478-00fa-4dbb-be30-185bd3fb746a «Documentation»,
 Bicycle «Classifier»,
 Wheel «Classifier»,
 BikeFork «Classifier»]

In [16]:
packages[0].throughOwningMembership[2].basic_name

'Wheel'

In [17]:
generate_values_for_type_annex_a(
    without_connectors_data,
    packages[0],
    packages[1],
    packages[0].throughOwningMembership[1],
    0,
    {}
)

Applying Annex A atom algorithm to Bicycle
Executing step 1. Working from Bicycle to create MyBicycle
...Found features [rollsOn: Wheel «Feature», holdsWheel: BikeFork «Feature»] under the type to value Bicycle.
Currently on pass Non-connector Features under Bicycle «Classifier».
Current values dict at this step is

...Found effective lower multiplicity of rollsOn (ba23b9d1-762e-4e40-8faa-8037ed7fc476) as 2.
...Looking to see if rollsOn: Wheel «Feature» is bound to other feature values
...Feature values for rollsOn: Wheel «Feature» are []
Executing step 2. Identified rollsOn (ba23b9d1-762e-4e40-8faa-8037ed7fc476) as a non-connector Feature. No existing values found. Generating 2 new values specializing type Wheel «Classifier».
Executing step 3a. Creating atom #1 to be value for rollsOn and specializing Wheel
...No lower features found for rollsOn: Wheel «Feature». Will finish descent here.
Executing step 3a. Creating atom #2 to be value for rollsOn and specializing Wheel
...No lower fe

{'ba23b9d1-762e-4e40-8faa-8037ed7fc476': [MyWheel1 «Classifier»,
  MyWheel2 «Classifier»]}

In [18]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
classifier MyBicycle specializes Bicycle {
   feature rollsOn (Covered) : Values for rollsOn;
}

#atom
classifier MyWheel1 specializes Wheel;

#atom
classifier MyWheel2 specializes Wheel;

classifier Values for rollsOn unions MyWheel1, MyWheel2;



## Annex A.3.3 One-To-One Connectors Case

In [19]:
filename = "A-3-3-OneToOneConnectors"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

one_2_one_connectors_data = pm.Model.load_from_post_file(json_file)
one_2_one_connectors_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-3-OneToOneConnectors.json)>

In [20]:
packages = [ele for ele in one_2_one_connectors_data.elements.values() if ele._metatype == 'Package']
packages

[Atoms «Package»,
 WithoutConnectorsModelToBeExecuted «Package»,
 OneToOneConnectorsModelToBeExecuted «Package»,
 OneToOneConnectorsExecution «Package»]

In [21]:
packages[2].throughOwningMembership

[df0fadcc-1594-406f-b9ed-4c4380f41787 «Documentation»,
 Bicycle «Classifier»,
 <Association([Wheel «Classifier»] ←→ [BikeFork «Classifier»])>]

In [22]:
generate_values_for_type_annex_a(
    one_2_one_connectors_data,
    packages[2],
    packages[3],
    packages[2].throughOwningMembership[1],
    0,
    {}
)

Applying Annex A atom algorithm to Bicycle
Executing step 1. Working from Bicycle to create MyBicycle
...Found features [<Connector([rollsOn: Wheel «Feature»] ←→ [holdsWheel: BikeFork «Feature»])>, rollsOn: Wheel «Feature», holdsWheel: BikeFork «Feature»] under the type to value Bicycle.
Currently on pass Non-connector Features under Bicycle «Classifier».
Current values dict at this step is

...Found effective lower multiplicity of rollsOn (fb0065c5-0cdc-4f61-be2b-29c9083f5efe) as 2.
...Looking to see if rollsOn: Wheel «Feature» is bound to other feature values
...Feature values for rollsOn: Wheel «Feature» are []
Executing step 2. Identified rollsOn (fb0065c5-0cdc-4f61-be2b-29c9083f5efe) as a non-connector Feature. No existing values found. Generating 2 new values specializing type Wheel «Classifier».
Executing step 3a. Creating atom #1 to be value for rollsOn and specializing Wheel
...No lower features found for rollsOn: Wheel «Feature». Will finish descent here.
Executing step 3a. C

{'fb0065c5-0cdc-4f61-be2b-29c9083f5efe': [MyWheel1 «Classifier»,
  MyWheel2 «Classifier»],
 'c465e95b-7fae-458d-8b3f-13faa3cc2f2f': [MyBikeFork1 «Classifier»,
  MyBikeFork2 «Classifier»],
 'c72e81c1-cf09-47a3-9196-dde232b7f02f': [<Association([MyWheel1 «Classifier»] ←→ [MyBikeFork1 «Classifier»])>,
  <Association([MyWheel2 «Classifier»] ←→ [MyBikeFork2 «Classifier»])>]}

In [23]:
packages[3].throughOwningMembership[4].throughSubclassification

[<Association([Wheel «Classifier»] ←→ [BikeFork «Classifier»])>]

In [24]:
for item in packages[3].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
classifier MyBicycle specializes Bicycle {
   feature rollsOn (Covered) : Values for rollsOn;
   feature holdsWheel (Covered) : Values for holdsWheel;
   connector fixWheel0 : BikeWheelFixed1;
   connector fixWheel1 : BikeWheelFixed2;
}

#atom
classifier MyWheel1 specializes Wheel;

#atom
classifier MyWheel2 specializes Wheel;

#atom
classifier MyBikeFork1 specializes BikeFork;

#atom
association BikeWheelFixed1 specializes BikeWheelFixed {
   end feature wheel : MyWheel1 redefines wheel;
   end feature fixedTo : MyBikeFork1 redefines fixedTo;
}

#atom
classifier MyBikeFork2 specializes BikeFork;

#atom
association BikeWheelFixed2 specializes BikeWheelFixed {
   end feature wheel : MyWheel2 redefines wheel;
   end feature fixedTo : MyBikeFork2 redefines fixedTo;
}

classifier Values for rollsOn unions MyWheel1, MyWheel2;

classifier Values for holdsWheel unions MyBikeFork1, MyBikeFork2;



## Annex A.3.6 Timing for Behaviors, Sequences

In [25]:
filename = "A-3-6-Sequences"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

sequences_data = pm.Model.load_from_post_file(json_file)
sequences_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-6-Sequences.json)>

In [26]:
packages = [ele for ele in sequences_data.elements.values() if ele._metatype == 'Package']
packages

[SequencesModelToBeExecuted «Package», SequencesExecution «Package»]

In [27]:
packages[0].ownedElement

[]

In [28]:
packages[0].throughOwningMembership

[38b62785-6ce9-40d6-87a8-e0f556581f56 «Documentation»,
 Manufacture «Behavior»,
 Paint «Behavior»,
 Dry «Behavior»,
 Ship «Behavior»]

In [29]:
generate_values_for_type_annex_a(
    sequences_data,
    packages[0],
    packages[1],
    packages[0].throughOwningMembership[1],
    0,
    {}
)

Applying Annex A atom algorithm to Manufacture
Executing step 1. Working from Manufacture to create MyManufacture
...Found features [paint: Paint «Step», dry: Dry «Step», <Succession([paint: Paint «Step»] ←→ [dry: Dry «Step»])>, ship: Ship «Step», <Succession([dry: Dry «Step»] ←→ [ship: Ship «Step»])>] under the type to value Manufacture.
Currently on pass Non-connector Features under Manufacture «Behavior».
Current values dict at this step is

...Found effective lower multiplicity of paint (25ee82c7-0485-4fb8-8b72-bbd960872469) as 1.
...Looking to see if paint: Paint «Step» is bound to other feature values
...Feature values for paint: Paint «Step» are []
Executing step 2. Identified paint (25ee82c7-0485-4fb8-8b72-bbd960872469) as a non-connector Feature. No existing values found. Generating 1 new values specializing type Paint «Behavior».
Executing step 3a. Creating atom #1 to be value for paint and specializing Paint
...No lower features found for paint: Paint «Step». Will finish des

{'25ee82c7-0485-4fb8-8b72-bbd960872469': [MyPaint1 «Behavior»],
 'eb0d5f1a-994b-46cb-ba8c-e0aca66d0ce6': [MyDry1 «Behavior»],
 '33dd8c59-6935-4815-acca-a2d753a6f81f': [<Association([MyPaint1 «Behavior»] ←→ [MyDry1 «Behavior»])>],
 'dd9bd2db-0174-42d5-94e3-ad8fc46a3c13': [MyShip1 «Behavior»],
 'e281beb2-a35a-468d-bda0-a65dd4372371': [<Association([MyDry1 «Behavior»] ←→ [MyShip1 «Behavior»])>]}

In [30]:
packages[1]

SequencesExecution «Package»

In [31]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
behavior MyManufacture specializes Manufacture {
   feature paint (Covered) : MyPaint1;
   feature dry (Covered) : MyDry1;
   succession p_before_d0 : p_before_d1;
   feature ship (Covered) : MyShip1;
   succession d_before_s0 : d_before_s1;
}

#atom
behavior MyPaint1 specializes Paint;

#atom
behavior MyDry1 specializes Dry;

#atom
association p_before_d1 {
   end feature earlierOccurrence : MyPaint1;
   end feature laterOccurrence : MyDry1;
}

#atom
behavior MyShip1 specializes Ship;

#atom
association d_before_s1 {
   end feature earlierOccurrence : MyDry1;
   end feature laterOccurrence : MyShip1;
}



## Annex A.3.8 Feature Value Changes

In [32]:
filename = "A-3-8-ChangingFeatureValues"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()) / "annex_a_data" / filename

values_data = pm.Model.load_from_post_file(json_file)
values_data

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\dev_docs\core_algos\annex_a_data\A-3-8-ChangingFeatureValues.json)>

In [33]:
packages = [ele for ele in values_data.elements.values() if ele._metatype == 'Package']
packages

[ChangingFeatureValuesModelToBeExecuted «Package»,
 ChangingFeatureValuesExecution «Package»]

In [34]:
#atom_generation_annex_a(values_data, packages[0], packages[1])

In [35]:
packages[0].throughOwningMembership

[72f33ab9-7786-4ddd-962b-3dba4f1cf004 «Documentation»,
 Manufacture «Behavior»,
 Product «Structure»,
 Paint «Behavior»,
 Dry «Behavior»,
 Ship «Behavior»]

In [36]:
packages[0].throughOwningMembership[1].throughFeatureMembership[1].feature

[objectToPaint: Product «Feature»,
 painting: FeatureWritePerformance «Step»,
 <Succession([painting: FeatureWritePerformance «Step»] ←→ [painted: FeatureWritePerformance «Step»])>,
 painted: FeatureWritePerformance «Step»,
 objectToPaint: Product «Feature»]

In [37]:
packages[0].throughOwningMembership[1].throughFeatureMembership[1].feature[2].throughSubsetting[0]

happensBeforeLinks «Feature»

In [38]:
generate_values_for_type_annex_a(
    values_data,
    packages[0],
    packages[1],
    packages[0].throughOwningMembership[1],
    0,
    {}
)

Applying Annex A atom algorithm to Manufacture
Executing step 1. Working from Manufacture to create MyManufacture
...Found features [objectToFinish: Product «Feature», paint: Paint «Step», dry: Dry «Step», <Succession([paint: Paint «Step»] ←→ [dry: Dry «Step»])>, ship: Ship «Step», <Succession([dry: Dry «Step»] ←→ [ship: Ship «Step»])>] under the type to value Manufacture.
Currently on pass Non-connector Features under Manufacture «Behavior».
Current values dict at this step is

...Found effective lower multiplicity of objectToFinish (62e2dc0b-a9e9-4097-aa48-209ef95bb612) as 1.
...Looking to see if objectToFinish: Product «Feature» is bound to other feature values
...Feature values for objectToFinish: Product «Feature» are []
Executing step 2. Identified objectToFinish (62e2dc0b-a9e9-4097-aa48-209ef95bb612) as a non-connector Feature. No existing values found. Generating 1 new values specializing type Product «Structure».
Executing step 3a. Creating atom #1 to be value for objectToFini

{'62e2dc0b-a9e9-4097-aa48-209ef95bb612': [MyProduct1 «Structure»],
 'dd76a0b4-2264-4cea-bd0f-69ebe307e1e4': [MyPaint1 «Behavior»],
 '952c4a56-d3c3-448a-b01a-2b2b70179ec2': [MyProduct1 «Structure»],
 'a61a7262-2293-473c-8b0c-ab8750d2a131': [MyProduct «Structure»,
  ProductTimeSlice «Structure»],
 '95bb916d-2254-4152-be1e-b767291341c7': [FeatureWritePerformanceForPainting «Behavior»],
 'cd6c3ed6-81f5-43bb-90b6-acaec9a69b7d': [FeatureWritePerformanceForPainted «Behavior»],
 '87b6e42e-dbf2-4918-923f-327496775f3f': [<Association([FeatureWritePerformanceForPainting «Behavior»] ←→ [FeatureWritePerformanceForPainted «Behavior»])>],
 '23a88d58-03fd-42a1-abce-b40e87f2f3cd': [MyDry1 «Behavior»],
 '65c2a9ca-a694-44ef-9e8d-780ae3eff90c': [MyProduct1 «Structure»],
 'd8c4f639-83f1-4913-b7d3-3b622c78600f': [MyProduct «Structure»,
  ProductTimeSlice «Structure»],
 '4f60b25a-70db-469b-adb4-575584db8a9f': [FeatureWritePerformanceForDried «Behavior»],
 '7076d315-a7cb-4a72-8451-d57a92c969f8': [<Association

In [39]:
for item in packages[1].throughOwningMembership:
    print(serialize_kerml_atom(item))

#atom
behavior MyManufacture specializes Manufacture {
   feature objectToFinish (Covered) : MyProduct1;
   feature paint (Covered) : MyPaint1;
   succession p_before_p0 : p_before_p1;
   feature dry (Covered) : MyDry1;
   succession p_before_d0 : p_before_d1;
   feature ship (Covered) : MyShip1;
   succession d_before_s0 : d_before_s1;
}

#atom
struct MyProduct1 specializes Product;

#atom
behavior MyPaint1 specializes Paint {
   feature objectToPaint (Covered) : MyProduct1;
   succession p_before_p0 : p_before_p1;
}

#atom
struct MyProduct specializes Product;

#atom
struct ProductTimeSlice {
   feature isPainted : Boolean redefines isPainted;
   feature isDry : Boolean redefines isDry;
   feature isShipped : Boolean redefines isShipped;
}

#atom
behavior FeatureWritePerformanceForPainting specializes FeatureWritePerformance;

#atom
behavior FeatureWritePerformanceForPainted specializes FeatureWritePerformance;

#atom
association p_before_p1 {
   end feature earlierOccurrence : Featu